Place where we do benchmarks to see how various code impacts performance

In [1]:
import os
import time
import numpy as np
from dataloader import DataLoader

In [2]:
dirname = os.getcwd()
loader = DataLoader(
    os.path.join(dirname, '../images_training_rev1'),
    os.path.join(dirname, '../training_solutions_rev1.csv')
)
batch_size = 500

In [3]:
soln = loader.load_all_solutions()

In [4]:
start_time = time.time()
image_batch = loader.load_images(soln[:batch_size, 0].astype(int))
print('Batch load {} images time {:.2f}s'.format(batch_size, time.time() - start_time))

print('{} images (RBG byte format) memory footprint {:.1f}MB'.format(
    batch_size,
    (image_batch.size * image_batch.itemsize) / 1e6
))

Batch load 500 images time 1.07s
500 images (RBG byte format) memory footprint 269.7MB


In [5]:
from transforms import normalize_images, color_index, combine

start_time = time.time()
preprocessor = combine(normalize_images, color_index)
image_norm_batch = preprocessor(image_batch)
print(image_norm_batch.shape)
print('Preprocessing took {:.2f}s'.format(time.time() - start_time))

print('{} images (preprocessed) memory footprint {:.1f}MB'.format(
    batch_size,
    (image_norm_batch.size * image_norm_batch.itemsize) / 1e6
))

(500, 4, 424, 424)
Preprocessing took 4.43s
500 images (preprocessed) memory footprint 2876.4MB


In [6]:
import torch
from torch.autograd import Variable
from pytorch_preprocess import NormalizeImages, ColorIndex, Combine

torch.cuda.synchronize()
start_time = time.time()

image_batch_var = Variable(torch.from_numpy(image_batch).type(torch.cuda.FloatTensor))
preprocessor = Combine(NormalizeImages(), ColorIndex())
image_norm_batch_torch = preprocessor(image_batch_var)

torch.cuda.synchronize()

print(image_norm_batch_torch.shape)
print('Preprocessing took {:.2f}s'.format(time.time() - start_time))
print('{} images (preprocessed) memory footprint {:.1f}MB'.format(
    batch_size,
    (image_norm_batch_torch.nelement() * image_norm_batch_torch.element_size()) / 1e6
))

torch.Size([500, 4, 424, 424])
Preprocessing took 0.35s
500 images (preprocessed) memory footprint 1438.2MB


In [8]:
rng = np.random.default_rng()

%timeit loader.load_images(soln[:batch_size, 0].astype(np.int))
%timeit loader.load_images(soln[:batch_size, 0].astype(np.int), rotations=rng.integers(0, 360, batch_size))
%timeit loader.load_images(soln[:batch_size, 0].astype(np.int), translations=rng.integers(-4, 4, (batch_size, 2), endpoint=True))
%timeit loader.load_images(soln[:batch_size, 0].astype(np.int), hflip=rng.choice([True, False], batch_size))

952 ms ± 4.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.08 s ± 2.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.01 s ± 2.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
986 ms ± 5.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
